## Using Flair Embedding and Glove over English dataset with 2 categories as a POC due to resources shortage

In [51]:
import pandas as pd
dataset = pd.read_csv('/home/nesma/SemesterII/Neural Networks/Project/multilingual-text-categorization-dataset/data/dataset.csv', sep='\t', header=None).applymap(str)
dataset.columns = ["Language","label","text"]
languagesData=[]
loc = 0
languages = dataset[dataset.columns[0]].unique()
for i in languages:
    name = languages[loc]+"Data" 
    globals()[name] = pd.DataFrame( dataset[dataset.Language == i])
    loc += 1

In [52]:
len(englishData)
englishData = englishData[1:180]


In [8]:
def lower_words(text):
    text = text.str.lower()
    return text

englishData['text'] = lower_words(englishData['text'])
input_str = englishData['text']
print(englishData.head())


   Language        label                                               text
6   english  advertising  musk bid for tesla: no formal offer, no firm d...
7   english  advertising  wall st. rallies on solid earnings, u.s.-china...
8   english  advertising  iran says no opec member can take over its sha...
9   english  advertising  online courses in data science that could help...
10  english  advertising  vw's ceo was told about emissions software mon...


In [9]:

import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
def remove_url(text):
    text = text.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
    return text


englishData['text'] = remove_url(englishData['text'])
englishData.head()

,Language,label,text
6,english,advertising,"musk bid for tesla: no formal offer, no firm d..."
7,english,advertising,"wall st. rallies on solid earnings, u.s.-china..."
8,english,advertising,iran says no opec member can take over its sha...
9,english,advertising,online courses in data science that could help...
10,english,advertising,vw's ceo was told about emissions software mon...


In [10]:
def remove_numbers(text):
    text = text.str.replace('\d+', '')
    return text


englishData['text'] = remove_numbers(englishData['text'])
englishData.head()

,Language,label,text
6,english,advertising,"musk bid for tesla: no formal offer, no firm d..."
7,english,advertising,"wall st. rallies on solid earnings, u.s.-china..."
8,english,advertising,iran says no opec member can take over its sha...
9,english,advertising,online courses in data science that could help...
10,english,advertising,vw's ceo was told about emissions software mon...


In [11]:
def remove_punctuations(text):
    text = text.str.replace('[^\w\s]','')
    return text


englishData['text'] = remove_punctuations(englishData['text'])
englishData.head()

,Language,label,text
6,english,advertising,musk bid for tesla no formal offer no firm dea...
7,english,advertising,wall st rallies on solid earnings uschina trad...
8,english,advertising,iran says no opec member can take over its sha...
9,english,advertising,online courses in data science that could help...
10,english,advertising,vws ceo was told about emissions software mont...


In [12]:

def remove_blank_space(col):
    col = col.str.strip()
    col = col.replace('\s+', ' ', regex=True)   
    return col


englishData['text'] = remove_blank_space(englishData.text)
englishData.head()

,Language,label,text
6,english,advertising,musk bid for tesla no formal offer no firm dea...
7,english,advertising,wall st rallies on solid earnings uschina trad...
8,english,advertising,iran says no opec member can take over its sha...
9,english,advertising,online courses in data science that could help...
10,english,advertising,vws ceo was told about emissions software mont...


In [13]:
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
def remove_stop_words(text, stop):    
    text.apply(lambda x: [item for item in x if item not in stop])
    return text

stop = stopwords.words('english')
remove_stop_words(englishData['text'],stop)
englishData.head()

Using TensorFlow backend.


,Language,label,text
6,english,advertising,musk bid for tesla no formal offer no firm dea...
7,english,advertising,wall st rallies on solid earnings uschina trad...
8,english,advertising,iran says no opec member can take over its sha...
9,english,advertising,online courses in data science that could help...
10,english,advertising,vws ceo was told about emissions software mont...


In [14]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


porter_stemmer = PorterStemmer()
englishData['text'] = englishData['text'].apply(stem_sentences)
englishData.head()

,Language,label,text
6,english,advertising,musk bid for tesla no formal offer no firm dea...
7,english,advertising,wall st ralli on solid earn uschina trade talk...
8,english,advertising,iran say no opec member can take over it share...
9,english,advertising,onlin cours in data scienc that could help you...
10,english,advertising,vw ceo wa told about emiss softwar month befor...


In [15]:
englishData = englishData[["label","text"]]
englishData['label'] = '__label__' + englishData['label'].astype(str)


In [16]:

englishData.iloc[0:int(len(englishData)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.8):int(len(englishData)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [17]:
englishData.head()

,label,text
6,__label__advertising,musk bid for tesla no formal offer no firm dea...
7,__label__advertising,wall st ralli on solid earn uschina trade talk...
8,__label__advertising,iran say no opec member can take over it share...
9,__label__advertising,onlin cours in data scienc that could help you...
10,__label__advertising,vw ceo wa told about emiss softwar month befor...


In [28]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings,DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


2019-05-11 18:43:46,029 Reading data from .
2019-05-11 18:43:46,030 Train: train.csv
2019-05-11 18:43:46,031 Dev: dev.csv
2019-05-11 18:43:46,031 Test: test.csv


In [19]:
len(corpus.dev)

18

In [38]:
word_embeddings = [WordEmbeddings('glove'),FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

In [39]:
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)


In [40]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)


In [41]:
trainer = ModelTrainer(classifier, corpus)


In [42]:
trainer.train('./', max_epochs=10, patience=1)

2019-05-11 19:00:36,474 ----------------------------------------------------------------------------------------------------
2019-05-11 19:00:36,475 Evaluation method: MICRO_F1_SCORE
2019-05-11 19:00:36,477 ----------------------------------------------------------------------------------------------------
2019-05-11 19:00:52,839 epoch 1 - iter 0/5 - loss 0.02151012
2019-05-11 19:01:22,865 epoch 1 - iter 1/5 - loss 0.02015468
2019-05-11 19:01:48,748 epoch 1 - iter 2/5 - loss 0.02130021
2019-05-11 19:02:08,466 epoch 1 - iter 3/5 - loss 0.02173255
2019-05-11 19:02:12,744 epoch 1 - iter 4/5 - loss 0.02510016
2019-05-11 19:02:12,766 ----------------------------------------------------------------------------------------------------
2019-05-11 19:02:12,767 EPOCH 1 done: loss 0.0251 - lr 0.1000 - bad epochs 0
2019-05-11 19:02:13,656 DEV  : loss 0.05333441 - f-score 0.0000 - acc 0.0000
2019-05-11 19:02:14,340 TEST : loss 0.05473531 - f-score 0.0000 - acc 0.0000
2019-05-11 19:02:17,402 -------

2019-05-11 19:16:35,440 loading file best-model.pt
2019-05-11 19:16:37,281 MICRO_AVG: acc 0.5 - f1-score 0.6667
2019-05-11 19:16:37,282 MACRO_AVG: acc 0.3333 - f1-score 0.4
2019-05-11 19:16:37,283 advertising tp: 0 - fp: 6 - fn: 0 - tn: 12 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-11 19:16:37,284 agriculture tp: 12 - fp: 0 - fn: 6 - tn: 0 - precision: 1.0000 - recall: 0.6667 - accuracy: 0.6667 - f1-score: 0.8000
2019-05-11 19:16:37,285 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6667,
 'dev_score_history': [0.0,
  0.0,
  0.0,
  0.2222,
  0.9444,
  0.7778,
  0.8333,
  0.7778,
  0.7778,
  0.0],
 'train_loss_history': [0.025100157103117776,
  0.022527735899476445,
  0.022367027314270243,
  0.021308427786125857,
  0.01984099320629064,
  0.02136232199914315,
  0.017950350966523674,
  0.016738595550551134,
  0.01566815420108683,
  0.02022500686785754],
 'dev_loss_history': [0.05333441123366356,
  0.09831058233976364,
  0.0568111352622509,
  0.05202953889966011,
  0.018089687451720238,
  0.02917196974158287,
  0.022702686488628387,
  0.033552512526512146,
  0.03192972391843796,
  0.16842232644557953]}